In [ ]:
library("Eunomia")
library("DatabaseConnector")



In [ ]:
conn <- DatabaseConnector::connect(dbms = "sqlite", server = getDatabaseFile("MIMIC"))

In [ ]:
sql <- "select count(*) from person;"
querySql(conn, sql)

In [ ]:
#options(databaseConnectorInteger64AsNumeric = FALSE)
connectionDetails <- getEunomiaConnectionDetails()
createCohorts(connectionDetails)
connection <- DatabaseConnector::connect(connectionDetails)
DatabaseConnector::querySql(connection, sql="SELECT * FROM person limit 10;")
#  COUNT(*)
#1     2694

#DatabaseConnector::getTableNames(connection,databaseSchema = 'main')
#DatabaseConnector::disconnect(connection)

In [ ]:
library(CohortMethod)
nsaids <- c(1118084, 1124300) # celecoxib, diclofenac
covSettings <- createDefaultCovariateSettings(
  excludedCovariateConceptIds = nsaids,
  addDescendantsToExclude = TRUE)

# Load data:
cmData <- getDbCohortMethodData(
  connectionDetails = connectionDetails,
  cdmDatabaseSchema = "main",
  targetId = 1,
  comparatorId = 2,
  outcomeIds = 3,
  exposureDatabaseSchema = "main",
  exposureTable = "cohort",
  outcomeDatabaseSchema = "main",
  outcomeTable = "cohort",
  covariateSettings = covSettings)
summary(cmData)